In [ ]:
## This file implements neural networks before and after lasso selection for p0017kpresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p0017kpresabsSTCC_qual.csv')
df.shape

(253, 7082)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
      ..
248    2
249    1
250    0
251    2
252    2
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,AAATATCTACCTTTTAATTTATGTAACTACTATTAGTATGCATATTCATTAGTTTTACCAGGACCATTAATTACATAAGATGATTTAGACTCTCCTTTTT,AAATAAGTGTGTTAGCTATAAAAAGAGATGAATAAAAACAAATATATTATATTTGGAGGAAGCGCCATGCTCAAAAGAAGTTTATTATTTTTAACTGTTT,AAAGTAGTATAAATGTTGTTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTT,AAAGAGATGAATAAAAACAAATATATTATATTTGGAGGAAGCGCCATGCTCAAAAGAAGTTTATTATTTTTAACTGTTTTATTGTTATTATTCTCATTTT,AAACGACCATTCAAAAGAACACGCAAAAGAAACTATAAGAAAAACAAACGGTGCTATTCGGTCAGCTTTAGATGACGCATTATATGATGGACTTATTTTT,AAAATGGTTATAAGAGGGGGGTTATACATGGATTTTTAAAATTATCGCGAAGTCGAGCCCCTCCCCGTTCCCCAAGTATTTTGATCGCTTTTGATTTTTT,AAAAGATTAATTATTTTTAACTAAAAATTAAAGTTCAATTAGTGTTTTGATGAATTTGGTCTTGATAAATTGAAATGATCTAAAAAACGCTATAATTTTT,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,5,5,2
1,109,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,8,0
2,115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,5,2
3,120335,0,0,0,0,0,0,0,1,0,...,1,1,0,1,0,0,1,5,5,2
4,120337,0,0,0,0,0,0,0,1,0,...,1,1,0,1,0,0,1,5,5,2


In [6]:
df['pheno'].value_counts()

2    91
0    88
1    74
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 7081)

In [9]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA,...,AAATATCTACCTTTTAATTTATGTAACTACTATTAGTATGCATATTCATTAGTTTTACCAGGACCATTAATTACATAAGATGATTTAGACTCTCCTTTTT,AAATAAGTGTGTTAGCTATAAAAAGAGATGAATAAAAACAAATATATTATATTTGGAGGAAGCGCCATGCTCAAAAGAAGTTTATTATTTTTAACTGTTT,AAAGTAGTATAAATGTTGTTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTT,AAAGAGATGAATAAAAACAAATATATTATATTTGGAGGAAGCGCCATGCTCAAAAGAAGTTTATTATTTTTAACTGTTTTATTGTTATTATTCTCATTTT,AAACGACCATTCAAAAGAACACGCAAAAGAAACTATAAGAAAAACAAACGGTGCTATTCGGTCAGCTTTAGATGACGCATTATATGATGGACTTATTTTT,AAAATGGTTATAAGAGGGGGGTTATACATGGATTTTTAAAATTATCGCGAAGTCGAGCCCCTCCCCGTTCCCCAAGTATTTTGATCGCTTTTGATTTTTT,AAAAGATTAATTATTTTTAACTAAAAATTAAAGTTCAATTAGTGTTTTGATGAATTTGGTCTTGATAAATTGAAATGATCTAAAAAACGCTATAATTTTT,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,5,5,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8,8,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,5,2
3,0,0,0,0,0,0,0,1,0,0,...,1,1,0,1,0,0,1,5,5,2
4,0,0,0,0,0,0,0,1,0,0,...,1,1,0,1,0,0,1,5,5,2


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 7081) (253,)


In [11]:
############# Fully-Connected Neural Network ################

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [14]:
dat = pd.DataFrame(X_test.iloc[:,0])
dat['test'] = y_test

In [15]:
dat

,id,test
112,NRS027,1
207,NRS246,1
186,NRS218,2
88,CFBRSa70,2
156,NRS177,1
...,...,...
244,SR3585,0
99,GA50819,2
147,NRS161,2
49,CFBREBSa114,1


In [16]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [17]:
#### neural network on over-sampling data
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [18]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 3ms/step - loss: 1.8286 - accuracy: 0.3955 - val_loss: 1.7507 - val_accuracy: 0.3947
Epoch 2/100
177/177 [==============================] - 0s 595us/step - loss: 1.3423 - accuracy: 0.3898 - val_loss: 2.0357 - val_accuracy: 0.3421
Epoch 3/100
177/177 [==============================] - 0s 469us/step - loss: 1.3016 - accuracy: 0.4237 - val_loss: 1.8341 - val_accuracy: 0.4474
Epoch 4/100
177/177 [==============================] - 0s 475us/step - loss: 1.1671 - accuracy: 0.4520 - val_loss: 1.5888 - val_accuracy: 0.4737
Epoch 5/100
177/177 [==============================] - 0s 542us/step - loss: 1.0002 - accuracy: 0.5537 - val_loss: 1.6090 - val_accuracy: 0.4474
Epoch 6/100
177/177 [==============================] - 0s 643us/step - loss: 0.9303 - accuracy: 0.5932 - val_loss: 1.4015 - val_accuracy: 0.4737
Epoch 7/100
177/177 [==============================] - 0s 558us/step - loss: 0.9054 - a

Epoch 57/100
177/177 [==============================] - 0s 327us/step - loss: 0.3659 - accuracy: 0.8644 - val_loss: 2.1291 - val_accuracy: 0.5000
Epoch 58/100
177/177 [==============================] - 0s 353us/step - loss: 0.3278 - accuracy: 0.8927 - val_loss: 2.5510 - val_accuracy: 0.3947
Epoch 59/100
177/177 [==============================] - 0s 361us/step - loss: 0.3172 - accuracy: 0.8757 - val_loss: 2.4577 - val_accuracy: 0.4211
Epoch 60/100
177/177 [==============================] - 0s 643us/step - loss: 0.2883 - accuracy: 0.8701 - val_loss: 2.2560 - val_accuracy: 0.4342
Epoch 61/100
177/177 [==============================] - 0s 583us/step - loss: 0.2964 - accuracy: 0.8983 - val_loss: 2.6415 - val_accuracy: 0.4211
Epoch 62/100
177/177 [==============================] - 0s 404us/step - loss: 0.2803 - accuracy: 0.9096 - val_loss: 2.6270 - val_accuracy: 0.4079
Epoch 63/100
177/177 [==============================] - 0s 413us/step - loss: 0.2763 - accuracy: 0.9040 - val_loss: 2.3129 -

In [41]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

76/76 [==============================] - 0s 146us/step
test accuracy: 46.05%


In [20]:
pred = model1.predict_classes(X_test)
pred

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 2, 2, 2, 0, 1, 2, 2, 1, 0, 0,
       2, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 2, 2, 0, 2, 0, 0, 1, 0, 0, 0,
       2, 2, 1, 0, 1, 2, 2, 1, 0, 2, 1, 2, 0, 1, 1, 0, 0, 1, 0, 2, 1, 1,
       0, 2, 1, 1, 0, 0, 0, 0, 0, 0])

In [21]:
dat['pred'] = pred
dat

,id,test,pred
112,NRS027,1,0
207,NRS246,1,0
186,NRS218,2,0
88,CFBRSa70,2,0
156,NRS177,1,1
...,...,...,...
244,SR3585,0,0
99,GA50819,2,0
147,NRS161,2,0
49,CFBREBSa114,1,0


In [22]:
proba1 = model1.predict_proba(X_test)
dat_proba1 = pd.DataFrame(proba1)

In [23]:
dat_proba1

,0,1,2
0,0.984498,0.003820,0.011683
1,0.788082,0.154701,0.057217
2,0.737957,0.170649,0.091394
3,0.675271,0.005887,0.318842
4,0.020528,0.969990,0.009482
...,...,...,...
71,0.543581,0.044045,0.412374
72,0.980923,0.017977,0.001100
73,0.999027,0.000967,0.000006
74,0.838052,0.122710,0.039238


In [24]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [25]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p17ST.csv", index = False,
         header=None)

In [45]:
hist1 = model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 505us/step - loss: 0.1900 - accuracy: 0.9322 - val_loss: 2.1362 - val_accuracy: 0.4342
Epoch 2/100
177/177 [==============================] - 0s 421us/step - loss: 0.2072 - accuracy: 0.9266 - val_loss: 2.3325 - val_accuracy: 0.4737
Epoch 3/100
177/177 [==============================] - 0s 457us/step - loss: 0.1913 - accuracy: 0.9209 - val_loss: 1.8385 - val_accuracy: 0.4342
Epoch 4/100
177/177 [==============================] - 0s 411us/step - loss: 0.2013 - accuracy: 0.9435 - val_loss: 2.1156 - val_accuracy: 0.4605
Epoch 5/100
177/177 [==============================] - 0s 378us/step - loss: 0.1875 - accuracy: 0.9266 - val_loss: 2.3817 - val_accuracy: 0.4868
Epoch 6/100
177/177 [==============================] - 0s 380us/step - loss: 0.2032 - accuracy: 0.9209 - val_loss: 2.2680 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 351us/step - loss: 0.1810 -

Epoch 57/100
177/177 [==============================] - 0s 446us/step - loss: 0.1094 - accuracy: 0.9661 - val_loss: 2.9556 - val_accuracy: 0.4737
Epoch 58/100
177/177 [==============================] - 0s 515us/step - loss: 0.0958 - accuracy: 0.9661 - val_loss: 2.7513 - val_accuracy: 0.4868
Epoch 59/100
177/177 [==============================] - 0s 462us/step - loss: 0.0925 - accuracy: 0.9718 - val_loss: 2.8178 - val_accuracy: 0.4737
Epoch 60/100
177/177 [==============================] - 0s 337us/step - loss: 0.0975 - accuracy: 0.9605 - val_loss: 3.0080 - val_accuracy: 0.4605
Epoch 61/100
177/177 [==============================] - 0s 336us/step - loss: 0.1086 - accuracy: 0.9379 - val_loss: 3.0334 - val_accuracy: 0.4737
Epoch 62/100
177/177 [==============================] - 0s 324us/step - loss: 0.1030 - accuracy: 0.9548 - val_loss: 2.9800 - val_accuracy: 0.5132
Epoch 63/100
177/177 [==============================] - 0s 319us/step - loss: 0.0961 - accuracy: 0.9605 - val_loss: 3.0126 -

In [46]:
print('train accuracy: %.2f%%' % (np.mean(hist1.history['accuracy'])*100))

train accuracy: 95.86%


In [16]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [17]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS027,1,0,0.759813,0.001304,0.238883
1,p0017kpresabs_qual,NRS246,1,0,0.947819,0.009591,0.042591
2,p0017kpresabs_qual,NRS218,2,1,0.008693,0.989390,0.001916
3,p0017kpresabs_qual,CFBRSa70,2,0,0.813774,0.001482,0.184744
4,p0017kpresabs_qual,NRS177,1,1,0.000916,0.998926,0.000157
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS187,1,1,0.216843,0.568123,0.215034
604,p0040presabsSTCC_qual,SR1287,0,1,0.418802,0.553160,0.028038
605,p0040presabsSTCC_qual,CFBRSa50,0,0,0.948100,0.035031,0.016869
606,p0040presabsSTCC_qual,NRS196,2,2,0.000964,0.039095,0.959940


In [18]:
y_prob = df_proba[df_proba['phage']=='p0017kpresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[9.84497600e-01, 3.81966210e-03, 1.16827550e-02],
       [7.88081700e-01, 1.54701190e-01, 5.72170700e-02],
       [7.37956940e-01, 1.70648720e-01, 9.13943800e-02],
       [6.75271030e-01, 5.88705530e-03, 3.18841900e-01],
       [2.05277700e-02, 9.69990200e-01, 9.48202800e-03],
       [9.52440260e-01, 4.55336050e-02, 2.02612440e-03],
       [5.78992840e-01, 1.77872760e-03, 4.19228460e-01],
       [9.89311300e-01, 1.86408400e-08, 1.06887035e-02],
       [9.78816570e-01, 2.04767290e-02, 7.06694960e-04],
       [4.48757670e-04, 6.23674170e-02, 9.37183860e-01],
       [6.54357800e-01, 3.45035230e-01, 6.06996360e-04],
       [9.21064600e-04, 7.28172360e-01, 2.70906630e-01],
       [4.51788680e-02, 2.53711730e-02, 9.29449900e-01],
       [4.47370170e-01, 9.34465450e-02, 4.59183280e-01],
       [3.34240530e-01, 3.12528000e-01, 3.53231460e-01],
       [9.80747300e-01, 1.92189630e-02, 3.38107460e-05],
       [3.82750300e-03, 9.86360000e-01, 9.81258800e-03],
       [2.04085500e-03, 6.69315

In [19]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [20]:
ovo1 = rocauc_ovo(y_test, y_prob, average="macro", multi_class="ovo")
ovo1

0.5648119517167135

In [21]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [22]:
ovr1 = rocauc_ovr(y_test, y_prob, average="macro", multi_class="ovr")
ovr1

0.5648119517167135

In [23]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y)

In [24]:
dat2 = pd.DataFrame(X_test.iloc[:,0])
dat2['test'] = y_test

In [25]:
dat2

,id,test
178,NRS210,2
104,Grady1,0
79,CFBRSa29,2
66,CFBRSa03,0
9,217,1
...,...,...
25,BCH-SA-10,1
94,GA15,2
244,SR3585,0
227,NRS387,2


In [26]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [30]:
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [31]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model2.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 5.7287 - accuracy: 0.3446 - val_loss: 1.4654 - val_accuracy: 0.3816
Epoch 2/100
177/177 [==============================] - 0s 389us/step - loss: 3.0653 - accuracy: 0.4181 - val_loss: 1.2886 - val_accuracy: 0.3816
Epoch 3/100
177/177 [==============================] - 0s 384us/step - loss: 1.7815 - accuracy: 0.4972 - val_loss: 1.2566 - val_accuracy: 0.3816
Epoch 4/100
177/177 [==============================] - 0s 402us/step - loss: 1.4527 - accuracy: 0.4859 - val_loss: 1.2613 - val_accuracy: 0.3947
Epoch 5/100
177/177 [==============================] - 0s 420us/step - loss: 1.1768 - accuracy: 0.5311 - val_loss: 1.1635 - val_accuracy: 0.4868
Epoch 6/100
177/177 [==============================] - 0s 371us/step - loss: 0.9800 - accuracy: 0.5593 - val_loss: 1.1365 - val_accuracy: 0.4737
Epoch 7/100
177/177 [==============================] - 0s 355us/step - loss: 0.9622 - a

Epoch 57/100
177/177 [==============================] - 0s 377us/step - loss: 0.4099 - accuracy: 0.8475 - val_loss: 1.5894 - val_accuracy: 0.4211
Epoch 58/100
177/177 [==============================] - 0s 658us/step - loss: 0.4361 - accuracy: 0.8701 - val_loss: 1.5307 - val_accuracy: 0.4474
Epoch 59/100
177/177 [==============================] - 0s 689us/step - loss: 0.4507 - accuracy: 0.8475 - val_loss: 1.5307 - val_accuracy: 0.4605
Epoch 60/100
177/177 [==============================] - 0s 300us/step - loss: 0.4298 - accuracy: 0.8588 - val_loss: 1.6431 - val_accuracy: 0.4079
Epoch 61/100
177/177 [==============================] - 0s 325us/step - loss: 0.4461 - accuracy: 0.8418 - val_loss: 1.5611 - val_accuracy: 0.4079
Epoch 62/100
177/177 [==============================] - 0s 331us/step - loss: 0.4501 - accuracy: 0.7966 - val_loss: 1.5466 - val_accuracy: 0.4342
Epoch 63/100
177/177 [==============================] - 0s 422us/step - loss: 0.3723 - accuracy: 0.8757 - val_loss: 1.6251 -

In [70]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

76/76 [==============================] - 0s 153us/step
test accuracy: 43.42%


In [33]:
pred2 = model2.predict_classes(X_test)
pred2

array([0, 0, 0, 1, 0, 2, 0, 1, 2, 0, 1, 2, 0, 1, 1, 0, 1, 1, 2, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 2, 2, 1, 2, 2, 0, 2, 0, 0, 0, 1, 0, 2, 1, 2, 1,
       0, 1, 0, 1, 1, 1, 2, 1, 2, 2, 0, 0, 2, 0, 1, 0, 1, 2, 2, 0, 0, 0,
       0, 1, 0, 2, 0, 1, 1, 0, 2, 1])

In [34]:
dat2['pred'] = pred2
dat2

,id,test,pred
178,NRS210,2,0
104,Grady1,0,0
79,CFBRSa29,2,0
66,CFBRSa03,0,1
9,217,1,0
...,...,...,...
25,BCH-SA-10,1,1
94,GA15,2,1
244,SR3585,0,0
227,NRS387,2,2


In [35]:
proba2 = model2.predict_proba(X_test)
dat_proba2 = pd.DataFrame(proba2)

In [36]:
dat_proba2

,0,1,2
0,0.424648,0.408826,0.166526
1,0.718681,0.180438,0.100881
2,0.947249,0.009377,0.043374
3,0.107967,0.647848,0.244185
4,0.529560,0.360273,0.110167
...,...,...,...
71,0.332118,0.531638,0.136244
72,0.289990,0.652280,0.057730
73,0.615252,0.118302,0.266446
74,0.427122,0.029366,0.543513


In [37]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [38]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p17ST.csv", index = False,
         header=None)

In [74]:
hist2 = model2.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 458us/step - loss: 0.3917 - accuracy: 0.8418 - val_loss: 1.8322 - val_accuracy: 0.3947
Epoch 2/100
177/177 [==============================] - 0s 413us/step - loss: 0.5722 - accuracy: 0.8192 - val_loss: 1.8582 - val_accuracy: 0.4079
Epoch 3/100
177/177 [==============================] - 0s 363us/step - loss: 0.4528 - accuracy: 0.8475 - val_loss: 1.8289 - val_accuracy: 0.4342
Epoch 4/100
177/177 [==============================] - 0s 336us/step - loss: 0.4144 - accuracy: 0.8249 - val_loss: 1.7495 - val_accuracy: 0.4737
Epoch 5/100
177/177 [==============================] - 0s 410us/step - loss: 0.3830 - accuracy: 0.8588 - val_loss: 1.8195 - val_accuracy: 0.4474
Epoch 6/100
177/177 [==============================] - 0s 380us/step - loss: 0.4009 - accuracy: 0.8531 - val_loss: 1.8094 - val_accuracy: 0.4342
Epoch 7/100
177/177 [==============================] - 0s 361us/step - loss: 0.3751 -

Epoch 57/100
177/177 [==============================] - 0s 431us/step - loss: 0.3562 - accuracy: 0.8588 - val_loss: 1.9866 - val_accuracy: 0.3553
Epoch 58/100
177/177 [==============================] - 0s 450us/step - loss: 0.4635 - accuracy: 0.8588 - val_loss: 1.9444 - val_accuracy: 0.3684
Epoch 59/100
177/177 [==============================] - 0s 429us/step - loss: 0.4262 - accuracy: 0.8814 - val_loss: 2.0161 - val_accuracy: 0.3816
Epoch 60/100
177/177 [==============================] - 0s 324us/step - loss: 0.3738 - accuracy: 0.8644 - val_loss: 2.0320 - val_accuracy: 0.3553
Epoch 61/100
177/177 [==============================] - 0s 360us/step - loss: 0.3831 - accuracy: 0.8418 - val_loss: 1.9790 - val_accuracy: 0.3684
Epoch 62/100
177/177 [==============================] - 0s 388us/step - loss: 0.4453 - accuracy: 0.8701 - val_loss: 1.9871 - val_accuracy: 0.3553
Epoch 63/100
177/177 [==============================] - 0s 419us/step - loss: 0.3711 - accuracy: 0.8757 - val_loss: 2.0691 -

In [75]:
print('train accuracy: %.2f%%' % (np.mean(hist2.history['accuracy'])*100))

train accuracy: 87.12%


In [32]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [33]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS210,2,0,0.999887,0.000112,8.851192e-07
1,p0017kpresabs_qual,Grady1,0,0,0.625329,0.369782,4.889404e-03
2,p0017kpresabs_qual,CFBRSa29,2,0,0.999098,0.000269,6.335156e-04
3,p0017kpresabs_qual,CFBRSa03,0,0,0.647338,0.331796,2.086646e-02
4,p0017kpresabs_qual,217,1,0,0.613342,0.381903,4.754707e-03
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS265,1,1,0.025601,0.687962,2.864372e-01
604,p0040presabsSTCC_qual,NY439,2,2,0.161947,0.266501,5.715521e-01
605,p0040presabsSTCC_qual,CFBRSa05,0,0,0.652983,0.254172,9.284494e-02
606,p0040presabsSTCC_qual,NRS205,2,2,0.000927,0.033760,9.653131e-01


In [34]:
y_prob2 = df_proba2[df_proba2['phage']=='p0017kpresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[4.24647660e-01, 4.08826020e-01, 1.66526320e-01],
       [7.18681100e-01, 1.80437520e-01, 1.00881360e-01],
       [9.47249000e-01, 9.37690200e-03, 4.33740900e-02],
       [1.07966706e-01, 6.47847830e-01, 2.44185400e-01],
       [5.29560150e-01, 3.60272680e-01, 1.10167170e-01],
       [5.15713060e-02, 8.18200100e-02, 8.66608700e-01],
       [9.76539130e-01, 1.04538990e-02, 1.30070230e-02],
       [4.20736760e-01, 4.39833020e-01, 1.39430120e-01],
       [3.09566180e-04, 7.84324800e-02, 9.21257900e-01],
       [9.53557250e-01, 3.84975000e-02, 7.94523000e-03],
       [4.01180400e-01, 4.24578250e-01, 1.74241440e-01],
       [9.39308850e-03, 2.11539610e-01, 7.79067300e-01],
       [8.04686900e-01, 1.72121930e-01, 2.31912320e-02],
       [2.61865760e-01, 7.31533000e-01, 6.60125860e-03],
       [2.38813100e-01, 6.65672060e-01, 9.55148040e-02],
       [6.25298260e-01, 2.95344740e-01, 7.93570300e-02],
       [5.00576700e-04, 9.99377200e-01, 1.22274700e-04],
       [3.22207180e-01, 6.70551

In [35]:
ovo2 = rocauc_ovo(y_test, y_prob2, average="macro", multi_class="ovo")
ovo2

0.5709819281247853

In [36]:
ovr2 = rocauc_ovr(y_test, y_prob2, average="macro", multi_class="ovr")
ovr2

0.5709819281247853

In [37]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y)

In [38]:
dat3 = pd.DataFrame(X_test.iloc[:,0])
dat3['test'] = y_test

In [39]:
dat3

,id,test
231,NY360,2
92,EUH25,2
91,EUH15,2
203,NRS241,0
242,SR2852,2
...,...,...
26,BCH-SA-11,0
111,NRS022,1
96,GA27,2
129,NRS102,1


In [40]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [43]:
model3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [44]:
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
model3.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 2.3523 - accuracy: 0.2881 - val_loss: 3.2508 - val_accuracy: 0.4737
Epoch 2/100
177/177 [==============================] - 0s 400us/step - loss: 1.6915 - accuracy: 0.4633 - val_loss: 2.7861 - val_accuracy: 0.4474
Epoch 3/100
177/177 [==============================] - 0s 379us/step - loss: 1.3269 - accuracy: 0.5254 - val_loss: 2.2743 - val_accuracy: 0.3289
Epoch 4/100
177/177 [==============================] - 0s 360us/step - loss: 1.0844 - accuracy: 0.5141 - val_loss: 1.8620 - val_accuracy: 0.3026
Epoch 5/100
177/177 [==============================] - 0s 346us/step - loss: 1.0418 - accuracy: 0.5593 - val_loss: 1.5707 - val_accuracy: 0.3421
Epoch 6/100
177/177 [==============================] - 0s 378us/step - loss: 0.9628 - accuracy: 0.6384 - val_loss: 1.4432 - val_accuracy: 0.3947
Epoch 7/100
177/177 [==============================] - 0s 481us/step - loss: 0.8618 - a

Epoch 57/100
177/177 [==============================] - 0s 366us/step - loss: 0.3773 - accuracy: 0.8870 - val_loss: 2.4871 - val_accuracy: 0.4079
Epoch 58/100
177/177 [==============================] - 0s 482us/step - loss: 0.3356 - accuracy: 0.8983 - val_loss: 2.6041 - val_accuracy: 0.4079
Epoch 59/100
177/177 [==============================] - 0s 532us/step - loss: 0.4069 - accuracy: 0.8870 - val_loss: 2.6427 - val_accuracy: 0.3553
Epoch 60/100
177/177 [==============================] - 0s 357us/step - loss: 0.3879 - accuracy: 0.8983 - val_loss: 2.3989 - val_accuracy: 0.3816
Epoch 61/100
177/177 [==============================] - 0s 561us/step - loss: 0.3260 - accuracy: 0.8927 - val_loss: 2.4294 - val_accuracy: 0.3947
Epoch 62/100
177/177 [==============================] - 0s 537us/step - loss: 0.3018 - accuracy: 0.8870 - val_loss: 2.5242 - val_accuracy: 0.3816
Epoch 63/100
177/177 [==============================] - 0s 446us/step - loss: 0.3151 - accuracy: 0.8927 - val_loss: 2.4605 -

In [99]:
acc_test3 = model3.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test3*100))

76/76 [==============================] - 0s 167us/step
test accuracy: 40.79%


In [46]:
pred3 = model3.predict_classes(X_test)
pred3

array([1, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 1, 0, 0, 2,
       0, 2, 1, 2, 1, 0, 0, 0, 0, 0, 1, 2, 1, 1, 2, 2, 0, 1, 1, 1, 0, 2,
       0, 0, 2, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 2, 2, 0, 1, 2,
       2, 0, 2, 1, 1, 0, 2, 2, 0, 0])

In [47]:
dat3['pred'] = pred3
dat3

,id,test,pred
231,NY360,2,1
92,EUH25,2,0
91,EUH15,2,2
203,NRS241,0,0
242,SR2852,2,2
...,...,...,...
26,BCH-SA-11,0,0
111,NRS022,1,2
96,GA27,2,2
129,NRS102,1,0


In [48]:
proba3 = model3.predict_proba(X_test)
dat_proba3 = pd.DataFrame(proba3)

In [49]:
dat_proba3

,0,1,2
0,0.257059,0.641684,0.101257
1,0.894516,0.081192,0.024292
2,0.011563,0.120927,0.867510
3,0.495871,0.477288,0.026841
4,0.001500,0.001606,0.996894
...,...,...,...
71,0.724754,0.229334,0.045912
72,0.011559,0.001357,0.987085
73,0.002119,0.101259,0.896621
74,0.943227,0.045022,0.011751


In [50]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [51]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p17ST.csv", index = False,
         header=None)

In [103]:
hist3 = model3.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 410us/step - loss: 0.2053 - accuracy: 0.9322 - val_loss: 2.6494 - val_accuracy: 0.3816
Epoch 2/100
177/177 [==============================] - 0s 416us/step - loss: 0.2112 - accuracy: 0.9266 - val_loss: 2.6600 - val_accuracy: 0.3816
Epoch 3/100
177/177 [==============================] - 0s 408us/step - loss: 0.2328 - accuracy: 0.9322 - val_loss: 2.6248 - val_accuracy: 0.3947
Epoch 4/100
177/177 [==============================] - 0s 384us/step - loss: 0.2154 - accuracy: 0.9322 - val_loss: 2.7844 - val_accuracy: 0.3553
Epoch 5/100
177/177 [==============================] - 0s 383us/step - loss: 0.2131 - accuracy: 0.9153 - val_loss: 2.7444 - val_accuracy: 0.3816
Epoch 6/100
177/177 [==============================] - 0s 370us/step - loss: 0.2042 - accuracy: 0.9435 - val_loss: 2.6425 - val_accuracy: 0.3947
Epoch 7/100
177/177 [==============================] - 0s 388us/step - loss: 0.1881 -

Epoch 57/100
177/177 [==============================] - 0s 418us/step - loss: 0.2328 - accuracy: 0.9492 - val_loss: 3.3875 - val_accuracy: 0.3289
Epoch 58/100
177/177 [==============================] - 0s 352us/step - loss: 0.2115 - accuracy: 0.9266 - val_loss: 3.2435 - val_accuracy: 0.3684
Epoch 59/100
177/177 [==============================] - 0s 332us/step - loss: 0.2790 - accuracy: 0.9492 - val_loss: 3.2966 - val_accuracy: 0.3553
Epoch 60/100
177/177 [==============================] - 0s 362us/step - loss: 0.2474 - accuracy: 0.9548 - val_loss: 3.2845 - val_accuracy: 0.3553
Epoch 61/100
177/177 [==============================] - 0s 386us/step - loss: 0.1419 - accuracy: 0.9548 - val_loss: 3.2767 - val_accuracy: 0.3553
Epoch 62/100
177/177 [==============================] - 0s 396us/step - loss: 0.1996 - accuracy: 0.9548 - val_loss: 3.4239 - val_accuracy: 0.3553
Epoch 63/100
177/177 [==============================] - 0s 417us/step - loss: 0.1932 - accuracy: 0.9492 - val_loss: 3.2621 -

In [104]:
print('train accuracy: %.2f%%' % (np.mean(hist3.history['accuracy'])*100))

train accuracy: 95.01%


In [41]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [42]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,2,2.165526e-02,4.848140e-01,0.493531
1,p0017kpresabs_qual,EUH25,2,0,9.986388e-01,1.245148e-03,0.000116
2,p0017kpresabs_qual,EUH15,2,2,9.227520e-04,1.424882e-02,0.984828
3,p0017kpresabs_qual,NRS241,0,0,8.374333e-01,1.614128e-01,0.001154
4,p0017kpresabs_qual,SR2852,2,2,3.976981e-09,5.145955e-10,1.000000
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,BCH-SA-01,0,0,9.305407e-01,6.356251e-02,0.005897
604,p0040presabsSTCC_qual,504,0,1,4.476389e-02,8.577548e-01,0.097481
605,p0040presabsSTCC_qual,GA27,2,1,2.779456e-01,5.384378e-01,0.183617
606,p0040presabsSTCC_qual,NRS209,1,0,4.210180e-01,3.559393e-01,0.223043


In [43]:
y_prob3 = df_proba3[df_proba3['phage']=='p0017kpresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[2.57059160e-01, 6.41683640e-01, 1.01257280e-01],
       [8.94515500e-01, 8.11922100e-02, 2.42923310e-02],
       [1.15627300e-02, 1.20926790e-01, 8.67510500e-01],
       [4.95870600e-01, 4.77288220e-01, 2.68411770e-02],
       [1.49988130e-03, 1.60604310e-03, 9.96894000e-01],
       [2.25119780e-02, 1.68889820e-05, 9.77471100e-01],
       [1.80777150e-01, 3.95467820e-01, 4.23755000e-01],
       [5.77097000e-04, 6.26295500e-04, 9.98796600e-01],
       [6.49840530e-01, 3.19829500e-02, 3.18176500e-01],
       [9.98679460e-01, 4.78549370e-04, 8.41911650e-04],
       [1.58170970e-01, 4.03310000e-01, 4.38519060e-01],
       [5.33674600e-01, 3.87690720e-01, 7.86347500e-02],
       [9.18829560e-01, 7.69565550e-02, 4.21389330e-03],
       [7.33572840e-01, 7.72389700e-02, 1.89188150e-01],
       [7.15747500e-01, 1.97265350e-02, 2.64526000e-01],
       [9.06369800e-02, 5.53815400e-01, 3.55547580e-01],
       [4.41347150e-01, 5.04651960e-01, 5.40008360e-02],
       [3.40273860e-02, 2.42209

In [44]:
ovo3 = rocauc_ovo(y_test, y_prob3, average="macro", multi_class="ovo")
ovo3

0.5330372889896698

In [45]:
ovr3 = rocauc_ovr(y_test, y_prob3, average="macro", multi_class="ovr")
ovr3

0.5330372889896698

In [46]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y)

In [47]:
dat4 = pd.DataFrame(X_test.iloc[:,0])
dat4['test'] = y_test

In [48]:
dat4

,id,test
236,SR1129,2
31,CA105,2
155,NRS175,1
92,EUH25,2
122,NRS070,2
...,...,...
235,SR1065,0
24,BCH-SA-09,0
104,Grady1,0
213,NRS254,2


In [49]:
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)

In [56]:
model4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [57]:
model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [58]:
model4.fit(X_train, y_train,
          batch_size=64, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 1ms/step - loss: 1.5504 - accuracy: 0.3559 - val_loss: 1.3531 - val_accuracy: 0.3026
Epoch 2/100
177/177 [==============================] - 0s 441us/step - loss: 1.1503 - accuracy: 0.4520 - val_loss: 1.2867 - val_accuracy: 0.3421
Epoch 3/100
177/177 [==============================] - 0s 468us/step - loss: 0.9935 - accuracy: 0.5480 - val_loss: 1.2375 - val_accuracy: 0.3158
Epoch 4/100
177/177 [==============================] - 0s 471us/step - loss: 0.9298 - accuracy: 0.5819 - val_loss: 1.3074 - val_accuracy: 0.3026
Epoch 5/100
177/177 [==============================] - 0s 496us/step - loss: 0.8241 - accuracy: 0.6271 - val_loss: 1.3592 - val_accuracy: 0.3158
Epoch 6/100
177/177 [==============================] - 0s 461us/step - loss: 0.7915 - accuracy: 0.6893 - val_loss: 1.4034 - val_accuracy: 0.3553
Epoch 7/100
177/177 [==============================] - 0s 367us/step - loss: 0.7636 - a

Epoch 57/100
177/177 [==============================] - 0s 396us/step - loss: 0.2728 - accuracy: 0.8983 - val_loss: 2.6668 - val_accuracy: 0.4474
Epoch 58/100
177/177 [==============================] - 0s 363us/step - loss: 0.2667 - accuracy: 0.8870 - val_loss: 2.6968 - val_accuracy: 0.4342
Epoch 59/100
177/177 [==============================] - 0s 329us/step - loss: 0.2712 - accuracy: 0.8927 - val_loss: 2.7045 - val_accuracy: 0.4474
Epoch 60/100
177/177 [==============================] - 0s 344us/step - loss: 0.2566 - accuracy: 0.8870 - val_loss: 2.6801 - val_accuracy: 0.4342
Epoch 61/100
177/177 [==============================] - 0s 349us/step - loss: 0.2812 - accuracy: 0.9096 - val_loss: 2.7139 - val_accuracy: 0.4474
Epoch 62/100
177/177 [==============================] - 0s 400us/step - loss: 0.2451 - accuracy: 0.9096 - val_loss: 2.7718 - val_accuracy: 0.4342
Epoch 63/100
177/177 [==============================] - 0s 351us/step - loss: 0.2646 - accuracy: 0.9096 - val_loss: 2.7849 -

In [128]:
acc_test4 = model4.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test4*100))

76/76 [==============================] - 0s 162us/step
test accuracy: 43.42%


In [59]:
pred4 = model4.predict_classes(X_test)
pred4

array([2, 2, 0, 0, 2, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2,
       1, 0, 2, 2, 2, 0, 2, 2, 0, 2, 1, 0, 2, 1, 1, 1, 0, 0, 2, 0, 1, 0,
       1, 2, 0, 1, 1, 2, 2, 2, 0, 2, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 2, 2,
       0, 2, 2, 2, 2, 0, 2, 1, 2, 0])

In [60]:
dat4['pred'] = pred4
dat4

,id,test,pred
236,SR1129,2,2
31,CA105,2,2
155,NRS175,1,0
92,EUH25,2,0
122,NRS070,2,2
...,...,...,...
235,SR1065,0,0
24,BCH-SA-09,0,2
104,Grady1,0,1
213,NRS254,2,2


In [61]:
proba4 = model4.predict_proba(X_test)
dat_proba4 = pd.DataFrame(proba4)

In [62]:
dat_proba4

,0,1,2
0,0.005313,5.045130e-03,0.989642
1,0.317836,6.064754e-02,0.621516
2,0.741557,2.228121e-01,0.035630
3,0.999357,6.244136e-07,0.000643
4,0.000022,8.065955e-06,0.999970
...,...,...,...
71,0.960508,4.693716e-07,0.039492
72,0.009070,2.135354e-01,0.777394
73,0.407448,5.212504e-01,0.071302
74,0.029411,1.292708e-02,0.957662


In [63]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [64]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p17ST.csv", index = False,
         header=None)

In [132]:
hist4 = model4.fit(X_train, y_train,
          batch_size=64, epochs=100,
          validation_data=(X_test, y_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 408us/step - loss: 0.1877 - accuracy: 0.9435 - val_loss: 3.1428 - val_accuracy: 0.3947
Epoch 2/100
177/177 [==============================] - 0s 390us/step - loss: 0.2036 - accuracy: 0.9322 - val_loss: 3.1284 - val_accuracy: 0.4342
Epoch 3/100
177/177 [==============================] - 0s 353us/step - loss: 0.1926 - accuracy: 0.9548 - val_loss: 3.1405 - val_accuracy: 0.4211
Epoch 4/100
177/177 [==============================] - 0s 341us/step - loss: 0.1933 - accuracy: 0.9322 - val_loss: 3.1627 - val_accuracy: 0.4211
Epoch 5/100
177/177 [==============================] - 0s 365us/step - loss: 0.1808 - accuracy: 0.9435 - val_loss: 3.1504 - val_accuracy: 0.4342
Epoch 6/100
177/177 [==============================] - 0s 344us/step - loss: 0.1809 - accuracy: 0.9605 - val_loss: 3.1611 - val_accuracy: 0.4342
Epoch 7/100
177/177 [==============================] - 0s 361us/step - loss: 0.1821 -

Epoch 57/100
177/177 [==============================] - 0s 393us/step - loss: 0.1321 - accuracy: 0.9548 - val_loss: 3.7890 - val_accuracy: 0.4211
Epoch 58/100
177/177 [==============================] - 0s 376us/step - loss: 0.1302 - accuracy: 0.9605 - val_loss: 3.7672 - val_accuracy: 0.4211
Epoch 59/100
177/177 [==============================] - 0s 406us/step - loss: 0.1322 - accuracy: 0.9605 - val_loss: 3.7929 - val_accuracy: 0.4211
Epoch 60/100
177/177 [==============================] - 0s 355us/step - loss: 0.1250 - accuracy: 0.9605 - val_loss: 3.8179 - val_accuracy: 0.4211
Epoch 61/100
177/177 [==============================] - 0s 367us/step - loss: 0.1456 - accuracy: 0.9661 - val_loss: 3.8270 - val_accuracy: 0.4211
Epoch 62/100
177/177 [==============================] - 0s 338us/step - loss: 0.1317 - accuracy: 0.9605 - val_loss: 3.8854 - val_accuracy: 0.4211
Epoch 63/100
177/177 [==============================] - 0s 348us/step - loss: 0.1287 - accuracy: 0.9492 - val_loss: 3.8758 -

In [133]:
print('train accuracy: %.2f%%' % (np.mean(hist4.history['accuracy'])*100))

train accuracy: 95.37%


In [50]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [51]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,SR1129,2,2,4.821690e-07,0.000001,9.999983e-01
1,p0017kpresabs_qual,CA105,2,2,2.652370e-03,0.000049,9.972990e-01
2,p0017kpresabs_qual,NRS175,1,1,1.198157e-02,0.988018,5.232074e-09
3,p0017kpresabs_qual,EUH25,2,1,1.388957e-01,0.735954,1.251503e-01
4,p0017kpresabs_qual,NRS070,2,1,7.231966e-03,0.810317,1.824512e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS247,1,2,2.957360e-02,0.385730,5.846961e-01
604,p0040presabsSTCC_qual,NRS215,1,2,3.196635e-03,0.445057,5.517461e-01
605,p0040presabsSTCC_qual,SR4152,2,2,4.319404e-02,0.028984,9.278219e-01
606,p0040presabsSTCC_qual,NRS035,1,1,4.902312e-03,0.680427,3.146706e-01


In [52]:
y_prob4 = df_proba4[df_proba4['phage']=='p0017kpresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[5.31287540e-03, 5.04513040e-03, 9.89641960e-01],
       [3.17836100e-01, 6.06475440e-02, 6.21516350e-01],
       [7.41557360e-01, 2.22812120e-01, 3.56304980e-02],
       [9.99356800e-01, 6.24413600e-07, 6.42553400e-04],
       [2.23217430e-05, 8.06595500e-06, 9.99969600e-01],
       [6.75432800e-04, 1.52419580e-02, 9.84082640e-01],
       [6.06920160e-02, 5.48378050e-01, 3.90929900e-01],
       [9.95311740e-01, 7.11470140e-04, 3.97686330e-03],
       [2.92135360e-01, 5.92035650e-01, 1.15829020e-01],
       [2.95903080e-05, 9.99097000e-01, 8.73445400e-04],
       [9.57156500e-03, 9.80777700e-01, 9.65080400e-03],
       [5.52573500e-04, 9.99297740e-01, 1.49565650e-04],
       [1.86405870e-01, 6.74111960e-01, 1.39482230e-01],
       [2.41502320e-02, 9.74214550e-01, 1.63521020e-03],
       [1.92317600e-02, 9.78346050e-01, 2.42219730e-03],
       [1.92181570e-01, 7.66911570e-01, 4.09069600e-02],
       [1.44074020e-05, 9.99985200e-01, 3.16928800e-07],
       [1.32297320e-01, 1.48308

In [53]:
ovo4 = rocauc_ovo(y_test, y_prob4, average="macro", multi_class="ovo")
ovo4

0.5960426258045306

In [54]:
ovr4 = rocauc_ovr(y_test, y_prob4, average="macro", multi_class="ovr")
ovr4

0.5960426258045306

In [55]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.5662184486589248

In [56]:
np.std(ovos)

0.022445154653505908

In [57]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.5662184486589248

In [58]:
np.std(ovrs)

0.022445154653505908

In [134]:
accs = [acc_test1, acc_test2, acc_test3, acc_test4]

In [135]:
mean = np.mean(accs)
print('test accuracy mean: %.2f%%' % (mean*100))

test accuracy mean: 43.42%


In [136]:
std = np.std(accs)
print('test accuracy standard deviation:', std)

test accuracy standard deviation: 0.018608075961940144


In [137]:
accs_train = [np.mean(hist1.history['accuracy']), np.mean(hist2.history['accuracy']), np.mean(hist3.history['accuracy']),
             np.mean(hist4.history['accuracy'])]

In [138]:
mean_train = np.mean(accs_train)
print('train accuracy mean: %.2f%%' % (mean_train*100))

train accuracy mean: 93.34%


In [139]:
std_train = np.std(accs_train)
print('train accuracy standard deviation:', std_train)

train accuracy standard deviation: 0.036008056


In [68]:
############ Feature selection using lasso ##########

In [59]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [60]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
selection.fit(X.loc[:, X.columns != 'id'], y)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [61]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [62]:
X_train_features = np.vstack((names, X.loc[:, X.columns != 'id']))
X_train_features = pd.DataFrame(X_train_features)

In [63]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7080
selected features: 765


In [64]:
cols = sel_features.values
cols.reshape((1, -1))

array([[   0,    9,   11,   18,   23,   38,   39,   40,   55,   56,   58,
          60,   61,   64,   73,   79,   84,  107,  109,  130,  133,  149,
         151,  159,  161,  164,  167,  168,  171,  182,  199,  202,  211,
         225,  227,  267,  270,  273,  274,  276,  297,  310,  313,  320,
         322,  323,  338,  347,  352,  355,  357,  374,  378,  389,  394,
         395,  401,  412,  422,  430,  458,  459,  464,  471,  481,  487,
         491,  498,  519,  533,  535,  537,  540,  541,  551,  556,  558,
         586,  590,  601,  616,  626,  632,  637,  639,  641,  664,  667,
         690,  697,  709,  742,  745,  759,  760,  761,  762,  779,  781,
         785,  794,  796,  822,  833,  838,  850,  866,  874,  881,  885,
         887,  933,  955,  969,  982,  983,  993,  994,  997, 1017, 1027,
        1031, 1042, 1047, 1052, 1060, 1064, 1071, 1088, 1109, 1114, 1117,
        1127, 1129, 1135, 1141, 1151, 1158, 1162, 1181, 1184, 1193, 1201,
        1202, 1208, 1221, 1225, 1255, 

In [65]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG',
       'TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA',
       'TTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCCTCGGCAGGAT',
       'TTTTTTGCCAATTTTGAGGAGGGAACACATGAAAACACGTTGTTACGATGGTAAAAAATGGCAATATGAGTTTAAGCATGAAGGAAAAAGATACCGTAAG',
       'TTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCAA',
       'TTTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCG',
       'TTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGAA',
       'TTTTTTAGCTTCAATGATAGCAGACATTCAGACACCTCCATTTAAATTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAA',
       'TTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCC',
 

In [66]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']

In [67]:
df_sel['strain'] = X.iloc[:,0]

In [68]:
df_sel

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGA,TTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCCTCGGCAGGAT,TTTTTTGCCAATTTTGAGGAGGGAACACATGAAAACACGTTGTTACGATGGTAAAAAATGGCAATATGAGTTTAAGCATGAAGGAAAAAGATACCGTAAG,TTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCAA,TTTTTTAGTTCCTACGAATAAGACTTGTCCACCATCTTCTGAAACTTGTTTCAAGAAGTTGTATGCCTCGTCTACTTTTTTCACTGTTTTTTGTAAGTCG,TTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTTTAGCATTTAAAGAAAAACAAACATCTTAATAGGAGGAA,TTTTTTAGCTTCAATGATAGCAGACATTCAGACACCTCCATTTAAATTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAA,TTTTTGGTGCTTTTTTTAAAATTAATAAAAAAGCACTTTCTACCCACGGCAAAAAGTGCTTGAAAGATCATAGATCATCACGTTCAAGTCAATTTTAGCC,TTTTTGGCTATAATAATATTTCTTTCTTTGTTCATATTACTTACCTCCTTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAA,...,AAGTATTTTGCCTACAAACCATAATTGAATTAAATTCCAAATTAACTGCACAGTACCTTTCAGTATCATTACAATGCCGTCCCAAACGCCTCGCCAGTTT,AAGTATATGAGTGCTTTTAATGTTAAATATGATGATATTTTTTTAGGTAATGAATACGAAAATTTCGTATTTACGAACGATAAAAAGAAATCAATTATTT,AAGTAGTATAAATGTTGTTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTT,AAGCCATTTGGTACCTTTACCTATTTGATCTTTTGTCCAGTTAAATGCCGATGATGCACCGGATTTAATATCTTTCCACATAGTACCTATGCTAAATCTT,AACGTTAGGGCAATAAATGAAAGTAGTATAAATGTTGTTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTT,AAATTTCGGGTAGCTCGCCTACCCTTATTATTTTTT,ST,CC,pheno,strain
0,1,1,0,1,1,1,1,0,0,1,...,1,1,1,1,1,1,5,5,2,107
1,0,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,8,8,0,109
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,5,5,2,115
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,5,5,2,120335
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,5,5,2,120337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,1,0,0,1,1,1,0,0,0,1,...,1,0,1,1,1,1,5,5,2,SR4152
249,1,0,0,1,0,1,0,0,0,0,...,1,0,1,1,1,1,3812,5,1,SR4153
250,0,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,5,5,0,SR4155
251,1,0,0,1,1,1,0,0,0,1,...,1,0,1,1,1,1,5,5,2,SR4156


In [69]:
X_sel = df_sel.loc[:, df_sel.columns != 'pheno']
y_sel = df_sel['pheno']
print(X_sel.shape, y_sel.shape, df_sel.shape)

(253, 766) (253,) (253, 767)


In [70]:
df_sel['pheno'].value_counts()

2    91
0    88
1    74
Name: pheno, dtype: int64

In [71]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_sel)

In [72]:
dat5 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat5['test'] = y_sel_test

In [73]:
dat5

,strain,test
231,NY360,2
139,NRS113,0
4,120337,2
59,CFBREBSa127,1
96,GA27,2
...,...,...
236,SR1129,2
145,NRS152,2
86,CFBRSa66A,0
28,BCH-SA-13,1


In [74]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [81]:
#### neural network on over-sampling data
model_sel = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [82]:
model_sel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [83]:
model_sel.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 633us/step - loss: 3.3655 - accuracy: 0.3729 - val_loss: 3.9710 - val_accuracy: 0.3684
Epoch 2/100
177/177 [==============================] - 0s 102us/step - loss: 2.2967 - accuracy: 0.4237 - val_loss: 3.5113 - val_accuracy: 0.4211
Epoch 3/100
177/177 [==============================] - 0s 139us/step - loss: 1.7892 - accuracy: 0.4407 - val_loss: 2.4204 - val_accuracy: 0.3816
Epoch 4/100
177/177 [==============================] - 0s 173us/step - loss: 1.4818 - accuracy: 0.4237 - val_loss: 1.9650 - val_accuracy: 0.3816
Epoch 5/100
177/177 [==============================] - 0s 183us/step - loss: 1.1147 - accuracy: 0.5311 - val_loss: 1.4581 - val_accuracy: 0.3684
Epoch 6/100
177/177 [==============================] - 0s 180us/step - loss: 0.9650 - accuracy: 0.5311 - val_loss: 1.4075 - val_accuracy: 0.3816
Epoch 7/100
177/177 [==============================] - 0s 178us/step - loss: 0.9002 -

Epoch 57/100
177/177 [==============================] - 0s 542us/step - loss: 0.4910 - accuracy: 0.8362 - val_loss: 1.5102 - val_accuracy: 0.4605
Epoch 58/100
177/177 [==============================] - 0s 274us/step - loss: 0.4822 - accuracy: 0.8362 - val_loss: 1.5079 - val_accuracy: 0.4474
Epoch 59/100
177/177 [==============================] - 0s 232us/step - loss: 0.4813 - accuracy: 0.8305 - val_loss: 1.5170 - val_accuracy: 0.4211
Epoch 60/100
177/177 [==============================] - 0s 258us/step - loss: 0.4771 - accuracy: 0.8531 - val_loss: 1.5544 - val_accuracy: 0.4605
Epoch 61/100
177/177 [==============================] - 0s 148us/step - loss: 0.4758 - accuracy: 0.8531 - val_loss: 1.5402 - val_accuracy: 0.4342
Epoch 62/100
177/177 [==============================] - 0s 148us/step - loss: 0.5501 - accuracy: 0.8249 - val_loss: 1.5801 - val_accuracy: 0.4342
Epoch 63/100
177/177 [==============================] - 0s 153us/step - loss: 0.4769 - accuracy: 0.8362 - val_loss: 1.6114 -

In [194]:
acc_test_sel = model_sel.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel*100))

76/76 [==============================] - 0s 102us/step
over-sampling test accuracy: 44.74%


In [84]:
pred5 = model_sel.predict_classes(X_sel_test)
pred5

array([1, 0, 2, 1, 2, 0, 1, 1, 0, 1, 0, 2, 0, 0, 2, 1, 2, 0, 2, 0, 0, 2,
       2, 2, 0, 2, 0, 1, 1, 1, 1, 2, 0, 0, 1, 1, 1, 2, 2, 1, 1, 0, 0, 0,
       1, 1, 2, 0, 2, 0, 2, 2, 0, 2, 1, 0, 1, 2, 2, 0, 1, 0, 1, 1, 2, 1,
       0, 0, 0, 2, 2, 2, 2, 0, 0, 2])

In [85]:
dat5['pred'] = pred5
dat5

,strain,test,pred
231,NY360,2,1
139,NRS113,0,0
4,120337,2,2
59,CFBREBSa127,1,1
96,GA27,2,2
...,...,...,...
236,SR1129,2,2
145,NRS152,2,2
86,CFBRSa66A,0,0
28,BCH-SA-13,1,0


In [86]:
proba5 = model_sel.predict_proba(X_sel_test)
dat_proba5 = pd.DataFrame(proba5)

In [87]:
dat_proba5

,0,1,2
0,4.082175e-01,5.667442e-01,0.025038
1,6.002990e-01,8.451332e-02,0.315188
2,6.900149e-04,3.875969e-03,0.995434
3,3.411632e-01,5.275964e-01,0.131240
4,3.080659e-03,3.997216e-02,0.956947
...,...,...,...
71,1.661970e-02,1.623360e-01,0.821044
72,1.031432e-01,3.281982e-02,0.864037
73,4.695159e-01,4.196791e-01,0.110805
74,6.495815e-01,2.143886e-01,0.136030


In [88]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [89]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p17ST.csv", index = False,
         header=None)

In [198]:
hist_sel = model_sel.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 135us/step - loss: 0.3837 - accuracy: 0.8814 - val_loss: 1.8082 - val_accuracy: 0.4868
Epoch 2/100
177/177 [==============================] - 0s 145us/step - loss: 0.3817 - accuracy: 0.8814 - val_loss: 1.8500 - val_accuracy: 0.5132
Epoch 3/100
177/177 [==============================] - 0s 184us/step - loss: 0.3830 - accuracy: 0.8757 - val_loss: 1.9227 - val_accuracy: 0.4868
Epoch 4/100
177/177 [==============================] - 0s 169us/step - loss: 0.3832 - accuracy: 0.8701 - val_loss: 1.8923 - val_accuracy: 0.5132
Epoch 5/100
177/177 [==============================] - 0s 179us/step - loss: 0.3864 - accuracy: 0.8757 - val_loss: 1.9255 - val_accuracy: 0.5000
Epoch 6/100
177/177 [==============================] - 0s 159us/step - loss: 0.3685 - accuracy: 0.8814 - val_loss: 1.9057 - val_accuracy: 0.4474
Epoch 7/100
177/177 [==============================] - 0s 148us/step - loss: 0.3892 -

Epoch 57/100
177/177 [==============================] - 0s 132us/step - loss: 0.2757 - accuracy: 0.9040 - val_loss: 2.0862 - val_accuracy: 0.4079
Epoch 58/100
177/177 [==============================] - 0s 123us/step - loss: 0.2770 - accuracy: 0.9040 - val_loss: 2.2006 - val_accuracy: 0.4605
Epoch 59/100
177/177 [==============================] - 0s 121us/step - loss: 0.2763 - accuracy: 0.9040 - val_loss: 2.2100 - val_accuracy: 0.4737
Epoch 60/100
177/177 [==============================] - 0s 124us/step - loss: 0.2764 - accuracy: 0.8983 - val_loss: 2.1566 - val_accuracy: 0.4342
Epoch 61/100
177/177 [==============================] - 0s 116us/step - loss: 0.2688 - accuracy: 0.9096 - val_loss: 2.1767 - val_accuracy: 0.4211
Epoch 62/100
177/177 [==============================] - 0s 144us/step - loss: 0.2676 - accuracy: 0.9040 - val_loss: 2.2519 - val_accuracy: 0.4605
Epoch 63/100
177/177 [==============================] - 0s 150us/step - loss: 0.2645 - accuracy: 0.8983 - val_loss: 2.2733 -

In [199]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel.history['accuracy'])*100))

train accuracy: 89.42%


In [75]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [76]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NY360,2,1,3.848032e-03,0.996096,0.000056
1,p0017kpresabs_qual,NRS113,0,0,8.441349e-01,0.000015,0.155850
2,p0017kpresabs_qual,120337,2,2,1.466081e-02,0.005713,0.979626
3,p0017kpresabs_qual,CFBREBSa127,1,0,9.985392e-01,0.000350,0.001111
4,p0017kpresabs_qual,GA27,2,2,1.665667e-08,0.000005,0.999995
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa131,2,1,6.617341e-03,0.565843,0.427540
604,p0040presabsSTCC_qual,NRS112,2,2,1.055758e-03,0.045611,0.953333
605,p0040presabsSTCC_qual,BCH-SA-06,0,0,8.747076e-01,0.071162,0.054130
606,p0040presabsSTCC_qual,834N,2,2,2.593112e-06,0.000013,0.999985


In [77]:
y_prob5 = df_proba5[df_proba5['phage']=='p0017kpresabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[4.0821750e-01, 5.6674420e-01, 2.5038283e-02],
       [6.0029900e-01, 8.4513320e-02, 3.1518772e-01],
       [6.9001490e-04, 3.8759685e-03, 9.9543400e-01],
       [3.4116316e-01, 5.2759635e-01, 1.3124049e-01],
       [3.0806590e-03, 3.9972164e-02, 9.5694710e-01],
       [8.4786296e-01, 4.2923980e-03, 1.4784463e-01],
       [1.9999656e-01, 6.2002190e-01, 1.7998166e-01],
       [4.2852017e-01, 5.4023160e-01, 3.1248229e-02],
       [9.6596190e-01, 3.0850316e-03, 3.0953050e-02],
       [4.3896747e-01, 5.1290360e-01, 4.8128980e-02],
       [5.6695880e-01, 2.5329900e-01, 1.7974226e-01],
       [3.3941576e-06, 9.5819590e-08, 9.9999654e-01],
       [5.3895920e-01, 3.4594210e-01, 1.1509866e-01],
       [4.8135006e-01, 1.9806825e-01, 3.2058170e-01],
       [6.1941595e-04, 3.2777651e-03, 9.9610280e-01],
       [2.5841850e-01, 6.9607740e-01, 4.5504045e-02],
       [3.0063888e-01, 9.5712420e-02, 6.0364866e-01],
       [9.9520420e-01, 1.3208898e-03, 3.4749506e-03],
       [2.9688856e-01, 1.552

In [78]:
ovo5 = rocauc_ovo(y_sel_test, y_prob5, average="macro", multi_class="ovo")
ovo5

0.6160385029432649

In [79]:
ovr5 = rocauc_ovr(y_sel_test, y_prob5, average="macro", multi_class="ovr")
ovr5

0.6160385029432649

In [80]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_sel)

In [81]:
dat6 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat6['test'] = y_sel_test

In [82]:
dat6

,strain,test
44,CFBREBSa104,1
170,NRS199,2
197,NRS233,1
238,SR1746,0
172,NRS202,2
...,...,...
92,EUH25,2
217,NRS260,0
75,CFBRSa25,0
156,NRS177,1


In [83]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [94]:
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [95]:
model_sel2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 2ms/step - loss: 3.7129 - accuracy: 0.4068 - val_loss: 2.0674 - val_accuracy: 0.3421
Epoch 2/100
177/177 [==============================] - 0s 73us/step - loss: 1.8724 - accuracy: 0.4576 - val_loss: 1.5226 - val_accuracy: 0.3816
Epoch 3/100
177/177 [==============================] - 0s 74us/step - loss: 1.7144 - accuracy: 0.4181 - val_loss: 1.4585 - val_accuracy: 0.3684
Epoch 4/100
177/177 [==============================] - 0s 83us/step - loss: 1.6059 - accuracy: 0.3898 - val_loss: 1.3899 - val_accuracy: 0.4474
Epoch 5/100
177/177 [==============================] - 0s 101us/step - loss: 1.3971 - accuracy: 0.4350 - val_loss: 1.0841 - val_accuracy: 0.5000
Epoch 6/100
177/177 [==============================] - 0s 109us/step - loss: 1.1408 - accuracy: 0.4746 - val_loss: 1.2275 - val_accuracy: 0.3553
Epoch 7/100
177/177 [==============================] - 0s 91us/step - loss: 1.2663 - accur

177/177 [==============================] - 0s 114us/step - loss: 0.5876 - accuracy: 0.7797 - val_loss: 1.0799 - val_accuracy: 0.5132
Epoch 58/100
177/177 [==============================] - 0s 103us/step - loss: 0.6113 - accuracy: 0.7627 - val_loss: 1.0658 - val_accuracy: 0.5132
Epoch 59/100
177/177 [==============================] - 0s 138us/step - loss: 0.6001 - accuracy: 0.7571 - val_loss: 1.0763 - val_accuracy: 0.5132
Epoch 60/100
177/177 [==============================] - 0s 130us/step - loss: 0.5615 - accuracy: 0.8079 - val_loss: 1.0993 - val_accuracy: 0.4737
Epoch 61/100
177/177 [==============================] - 0s 94us/step - loss: 0.6144 - accuracy: 0.7910 - val_loss: 1.0779 - val_accuracy: 0.5263
Epoch 62/100
177/177 [==============================] - 0s 84us/step - loss: 0.5782 - accuracy: 0.7910 - val_loss: 1.0825 - val_accuracy: 0.5132
Epoch 63/100
177/177 [==============================] - 0s 177us/step - loss: 0.6180 - accuracy: 0.7740 - val_loss: 1.0831 - val_accuracy: 

In [231]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

76/76 [==============================] - 0s 110us/step
test accuracy: 51.32%


In [97]:
pred6 = model_sel2.predict_classes(X_sel_test)
pred6

array([2, 0, 0, 1, 2, 2, 2, 1, 2, 0, 1, 2, 0, 2, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 1, 0, 2,
       1, 0, 0, 0, 0, 1, 2, 0, 1, 0, 2, 1, 0, 2, 0, 0, 0, 0, 2, 2, 2, 1,
       2, 1, 0, 0, 2, 0, 0, 0, 1, 2])

In [98]:
dat6['pred'] = pred6
dat6

,strain,test,pred
44,CFBREBSa104,1,2
170,NRS199,2,0
197,NRS233,1,0
238,SR1746,0,1
172,NRS202,2,2
...,...,...,...
92,EUH25,2,0
217,NRS260,0,0
75,CFBRSa25,0,0
156,NRS177,1,1


In [100]:
proba6 = model_sel2.predict_proba(X_sel_test)
dat_proba6 = pd.DataFrame(proba6)

In [101]:
dat_proba6

,0,1,2
0,0.086054,0.142993,0.770953
1,0.395020,0.367852,0.237128
2,0.390421,0.324556,0.285023
3,0.292312,0.414531,0.293156
4,0.186511,0.156528,0.656961
...,...,...,...
71,0.881564,0.110491,0.007945
72,0.723630,0.223998,0.052371
73,0.622915,0.221427,0.155658
74,0.077259,0.808464,0.114277


In [102]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [103]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p17ST.csv", index = False,
         header=None)

In [235]:
hist_sel2 = model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 98us/step - loss: 0.4095 - accuracy: 0.8475 - val_loss: 1.2108 - val_accuracy: 0.4474
Epoch 2/100
177/177 [==============================] - 0s 87us/step - loss: 0.3981 - accuracy: 0.8418 - val_loss: 1.2094 - val_accuracy: 0.4474
Epoch 3/100
177/177 [==============================] - 0s 93us/step - loss: 0.3950 - accuracy: 0.8305 - val_loss: 1.2103 - val_accuracy: 0.4868
Epoch 4/100
177/177 [==============================] - 0s 147us/step - loss: 0.3912 - accuracy: 0.8362 - val_loss: 1.2156 - val_accuracy: 0.5000
Epoch 5/100
177/177 [==============================] - 0s 98us/step - loss: 0.4043 - accuracy: 0.8475 - val_loss: 1.2285 - val_accuracy: 0.4868
Epoch 6/100
177/177 [==============================] - 0s 133us/step - loss: 0.4132 - accuracy: 0.8418 - val_loss: 1.2289 - val_accuracy: 0.5132
Epoch 7/100
177/177 [==============================] - 0s 120us/step - loss: 0.4174 - acc

177/177 [==============================] - 0s 80us/step - loss: 0.3055 - accuracy: 0.9040 - val_loss: 1.3293 - val_accuracy: 0.4737
Epoch 58/100
177/177 [==============================] - 0s 102us/step - loss: 0.3016 - accuracy: 0.9040 - val_loss: 1.3260 - val_accuracy: 0.4605
Epoch 59/100
177/177 [==============================] - 0s 94us/step - loss: 0.3012 - accuracy: 0.9040 - val_loss: 1.3218 - val_accuracy: 0.5000
Epoch 60/100
177/177 [==============================] - 0s 107us/step - loss: 0.3037 - accuracy: 0.8927 - val_loss: 1.3152 - val_accuracy: 0.4868
Epoch 61/100
177/177 [==============================] - 0s 116us/step - loss: 0.2999 - accuracy: 0.8983 - val_loss: 1.3215 - val_accuracy: 0.5000
Epoch 62/100
177/177 [==============================] - 0s 108us/step - loss: 0.3054 - accuracy: 0.9096 - val_loss: 1.3385 - val_accuracy: 0.4737
Epoch 63/100
177/177 [==============================] - 0s 97us/step - loss: 0.3020 - accuracy: 0.8983 - val_loss: 1.3494 - val_accuracy: 0

In [236]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel2.history['accuracy'])*100))

train accuracy: 88.28%


In [84]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [85]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,CFBREBSa104,1,2,1.290039e-09,1.567630e-07,9.999999e-01
1,p0017kpresabs_qual,NRS199,2,2,2.856965e-05,2.843749e-03,9.971277e-01
2,p0017kpresabs_qual,NRS233,1,0,9.999446e-01,4.541289e-06,5.090974e-05
3,p0017kpresabs_qual,SR1746,0,2,6.282367e-02,7.075194e-04,9.364688e-01
4,p0017kpresabs_qual,NRS202,2,2,8.229589e-03,2.163908e-05,9.917488e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,GA27,2,1,3.964591e-03,9.959286e-01,1.068284e-04
604,p0040presabsSTCC_qual,GA231,2,1,3.309226e-04,9.996691e-01,6.232397e-10
605,p0040presabsSTCC_qual,SR1287,0,1,8.445997e-06,9.999915e-01,1.182947e-13
606,p0040presabsSTCC_qual,506,2,0,6.516150e-01,1.480882e-02,3.335762e-01


In [86]:
y_prob6 = df_proba6[df_proba6['phage']=='p0017kpresabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[8.60537700e-02, 1.42993090e-01, 7.70953200e-01],
       [3.95020370e-01, 3.67851530e-01, 2.37128170e-01],
       [3.90420730e-01, 3.24556300e-01, 2.85022940e-01],
       [2.92312440e-01, 4.14531230e-01, 2.93156300e-01],
       [1.86510850e-01, 1.56528200e-01, 6.56960900e-01],
       [1.59804400e-02, 2.01865700e-01, 7.82153840e-01],
       [1.33841780e-01, 1.47130460e-01, 7.19027760e-01],
       [7.79819800e-02, 8.24907660e-01, 9.71104300e-02],
       [3.05521400e-03, 4.45698500e-01, 5.51246300e-01],
       [4.39244960e-01, 4.20237930e-01, 1.40517100e-01],
       [1.24578830e-01, 5.27910700e-01, 3.47510460e-01],
       [1.68469100e-01, 2.80017050e-01, 5.51513900e-01],
       [6.63150800e-01, 1.69104840e-01, 1.67744350e-01],
       [5.31255600e-02, 1.10729300e-01, 8.36145160e-01],
       [1.01157980e-02, 9.48802100e-01, 4.10821300e-02],
       [6.06533350e-01, 1.99076850e-01, 1.94389780e-01],
       [1.34231050e-01, 6.58513400e-01, 2.07255450e-01],
       [2.54778560e-01, 3.91055

In [87]:
ovo6 = rocauc_ovo(y_sel_test, y_prob6, average="macro", multi_class="ovo")
ovo6

0.6597351061636776

In [88]:
ovr6 = rocauc_ovr(y_sel_test, y_prob6, average="macro", multi_class="ovr")
ovr6

0.6597351061636776

In [89]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_sel)

In [90]:
dat7 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat7['test'] = y_sel_test

In [91]:
dat7

,strain,test
222,NRS271,1
242,SR2852,2
34,CA39,2
221,NRS266,1
230,NY356,2
...,...,...
105,MN055,2
67,CFBRSa04,0
235,SR1065,0
190,NRS224,1


In [92]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [108]:
model_sel3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(3, activation='softmax'),
])

In [109]:
model_sel3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [110]:
model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 731us/step - loss: 5.9042 - accuracy: 0.4068 - val_loss: 2.5598 - val_accuracy: 0.4605
Epoch 2/100
177/177 [==============================] - 0s 110us/step - loss: 4.7050 - accuracy: 0.4633 - val_loss: 1.9082 - val_accuracy: 0.4079
Epoch 3/100
177/177 [==============================] - 0s 126us/step - loss: 3.5429 - accuracy: 0.4463 - val_loss: 1.5122 - val_accuracy: 0.3553
Epoch 4/100
177/177 [==============================] - 0s 179us/step - loss: 2.7681 - accuracy: 0.4237 - val_loss: 1.4511 - val_accuracy: 0.3816
Epoch 5/100
177/177 [==============================] - 0s 129us/step - loss: 2.1327 - accuracy: 0.4181 - val_loss: 1.2740 - val_accuracy: 0.4605
Epoch 6/100
177/177 [==============================] - 0s 183us/step - loss: 1.5930 - accuracy: 0.4463 - val_loss: 1.2814 - val_accuracy: 0.3684
Epoch 7/100
177/177 [==============================] - 0s 170us/step - loss: 1.1270 -

Epoch 57/100
177/177 [==============================] - 0s 136us/step - loss: 0.5476 - accuracy: 0.7797 - val_loss: 1.2773 - val_accuracy: 0.4474
Epoch 58/100
177/177 [==============================] - 0s 113us/step - loss: 0.5497 - accuracy: 0.7910 - val_loss: 1.2947 - val_accuracy: 0.4079
Epoch 59/100
177/177 [==============================] - 0s 140us/step - loss: 0.5395 - accuracy: 0.7797 - val_loss: 1.3499 - val_accuracy: 0.3947
Epoch 60/100
177/177 [==============================] - 0s 151us/step - loss: 0.5361 - accuracy: 0.7684 - val_loss: 1.3410 - val_accuracy: 0.3947
Epoch 61/100
177/177 [==============================] - 0s 99us/step - loss: 0.5327 - accuracy: 0.7910 - val_loss: 1.2984 - val_accuracy: 0.4211
Epoch 62/100
177/177 [==============================] - 0s 137us/step - loss: 0.5377 - accuracy: 0.7910 - val_loss: 1.3204 - val_accuracy: 0.3947
Epoch 63/100
177/177 [==============================] - 0s 152us/step - loss: 0.5270 - accuracy: 0.7966 - val_loss: 1.3381 - 

In [264]:
acc_test_sel3 = model_sel3.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel3*100))

76/76 [==============================] - 0s 120us/step
test accuracy: 42.11%


In [111]:
pred7 = model_sel3.predict_classes(X_sel_test)
pred7

array([1, 2, 0, 2, 1, 0, 1, 2, 2, 1, 0, 1, 2, 0, 0, 0, 2, 1, 1, 2, 2, 0,
       0, 0, 1, 2, 0, 0, 1, 0, 1, 0, 0, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 0,
       2, 1, 0, 1, 0, 2, 1, 2, 2, 0, 1, 0, 1, 1, 2, 2, 0, 0, 0, 1, 1, 2,
       2, 0, 1, 1, 0, 2, 1, 0, 0, 1])

In [112]:
dat7['pred'] = pred7
dat7

,strain,test,pred
222,NRS271,1,1
242,SR2852,2,2
34,CA39,2,0
221,NRS266,1,2
230,NY356,2,1
...,...,...,...
105,MN055,2,2
67,CFBRSa04,0,1
235,SR1065,0,0
190,NRS224,1,0


In [113]:
proba7 = model_sel3.predict_proba(X_sel_test)
dat_proba7 = pd.DataFrame(proba7)

In [114]:
dat_proba7

,0,1,2
0,0.025803,0.933269,0.040928
1,0.051309,0.398973,0.549717
2,0.392121,0.360336,0.247543
3,0.221286,0.216210,0.562504
4,0.460525,0.474938,0.064536
...,...,...,...
71,0.249112,0.327617,0.423271
72,0.416233,0.551626,0.032140
73,0.907585,0.000580,0.091835
74,0.641654,0.018258,0.340088


In [115]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [116]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p17ST.csv", index = False,
         header=None)

In [268]:
hist_sel3 = model_sel3.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 97us/step - loss: 0.4401 - accuracy: 0.8418 - val_loss: 1.4228 - val_accuracy: 0.4079
Epoch 2/100
177/177 [==============================] - 0s 85us/step - loss: 0.4439 - accuracy: 0.8362 - val_loss: 1.4283 - val_accuracy: 0.3947
Epoch 3/100
177/177 [==============================] - 0s 107us/step - loss: 0.4330 - accuracy: 0.8418 - val_loss: 1.4474 - val_accuracy: 0.3947
Epoch 4/100
177/177 [==============================] - 0s 124us/step - loss: 0.4353 - accuracy: 0.8418 - val_loss: 1.4709 - val_accuracy: 0.3947
Epoch 5/100
177/177 [==============================] - 0s 157us/step - loss: 0.4292 - accuracy: 0.8305 - val_loss: 1.4485 - val_accuracy: 0.4211
Epoch 6/100
177/177 [==============================] - 0s 94us/step - loss: 0.4320 - accuracy: 0.8418 - val_loss: 1.4680 - val_accuracy: 0.4211
Epoch 7/100
177/177 [==============================] - 0s 107us/step - loss: 0.4312 - ac

177/177 [==============================] - 0s 83us/step - loss: 0.3399 - accuracy: 0.8757 - val_loss: 1.6177 - val_accuracy: 0.3816
Epoch 58/100
177/177 [==============================] - 0s 92us/step - loss: 0.3386 - accuracy: 0.8701 - val_loss: 1.6827 - val_accuracy: 0.3684
Epoch 59/100
177/177 [==============================] - 0s 81us/step - loss: 0.3342 - accuracy: 0.8927 - val_loss: 1.6837 - val_accuracy: 0.3816
Epoch 60/100
177/177 [==============================] - 0s 88us/step - loss: 0.3398 - accuracy: 0.8814 - val_loss: 1.6628 - val_accuracy: 0.3684
Epoch 61/100
177/177 [==============================] - 0s 91us/step - loss: 0.3329 - accuracy: 0.8814 - val_loss: 1.7360 - val_accuracy: 0.3553
Epoch 62/100
177/177 [==============================] - 0s 91us/step - loss: 0.3413 - accuracy: 0.8870 - val_loss: 1.7151 - val_accuracy: 0.3684
Epoch 63/100
177/177 [==============================] - 0s 88us/step - loss: 0.3393 - accuracy: 0.8701 - val_loss: 1.6624 - val_accuracy: 0.381

In [269]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel3.history['accuracy'])*100))

train accuracy: 86.64%


In [93]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [94]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS271,1,0,0.854929,0.144940,0.000130
1,p0017kpresabs_qual,SR2852,2,2,0.000001,0.000075,0.999923
2,p0017kpresabs_qual,CA39,2,0,0.959992,0.013406,0.026602
3,p0017kpresabs_qual,NRS266,1,2,0.000018,0.000003,0.999980
4,p0017kpresabs_qual,NY356,2,2,0.009156,0.411991,0.578852
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,CFBREBSa110,2,2,0.044135,0.010244,0.945621
604,p0040presabsSTCC_qual,CFBRSa05,0,2,0.145032,0.408530,0.446439
605,p0040presabsSTCC_qual,CFBREBSa123,0,0,0.994623,0.004152,0.001224
606,p0040presabsSTCC_qual,NY360,1,1,0.000795,0.948566,0.050639


In [95]:
y_prob7 = df_proba7[df_proba7['phage']=='p0017kpresabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[2.5803387e-02, 9.3326920e-01, 4.0927507e-02],
       [5.1309418e-02, 3.9897350e-01, 5.4971720e-01],
       [3.9212075e-01, 3.6033604e-01, 2.4754323e-01],
       [2.2128580e-01, 2.1620992e-01, 5.6250435e-01],
       [4.6052542e-01, 4.7493818e-01, 6.4536430e-02],
       [8.2702476e-01, 4.3616660e-02, 1.2935860e-01],
       [1.2552959e-01, 7.9958440e-01, 7.4885994e-02],
       [2.4747360e-01, 1.1413598e-01, 6.3839040e-01],
       [1.9762398e-01, 2.9876280e-01, 5.0361323e-01],
       [2.5151002e-01, 7.2410260e-01, 2.4387326e-02],
       [9.5353264e-01, 2.8650573e-04, 4.6180807e-02],
       [1.9091031e-03, 9.4689300e-01, 5.1197970e-02],
       [4.6845627e-04, 3.8345692e-01, 6.1607470e-01],
       [9.4751570e-01, 1.5126044e-03, 5.0971690e-02],
       [5.9009504e-01, 3.6448804e-01, 4.5416960e-02],
       [6.0963714e-01, 3.0628765e-01, 8.4075265e-02],
       [3.2645352e-02, 6.5103010e-02, 9.0225166e-01],
       [3.1774396e-01, 4.9754880e-01, 1.8470727e-01],
       [3.6089970e-01, 3.986

In [96]:
ovo7 = rocauc_ovo(y_sel_test, y_prob7, average="macro", multi_class="ovo")
ovo7

0.5956017086969468

In [97]:
ovr7 = rocauc_ovr(y_sel_test, y_prob7, average="macro", multi_class="ovr")
ovr7

0.5956017086969468

In [98]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_sel)

In [99]:
dat8 = pd.DataFrame(X_sel_test.iloc[:,-1])
dat8['test'] = y_sel_test

In [100]:
dat8

,strain,test
131,NRS104,0
123,NRS071,0
124,NRS072,1
27,BCH-SA-12,0
57,CFBREBSa125,2
...,...,...
170,NRS199,2
32,CA11,2
212,NRS253,1
141,NRS119,2


In [101]:
X_sel_train = X_sel_train.drop(['strain'], axis=1)
X_sel_test = X_sel_test.drop(['strain'], axis=1)

In [102]:
model_sel4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [122]:
model_sel4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [123]:
model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 858us/step - loss: 1.6399 - accuracy: 0.4124 - val_loss: 1.4864 - val_accuracy: 0.2105
Epoch 2/100
177/177 [==============================] - 0s 193us/step - loss: 1.2767 - accuracy: 0.4237 - val_loss: 1.1999 - val_accuracy: 0.3947
Epoch 3/100
177/177 [==============================] - 0s 172us/step - loss: 1.1525 - accuracy: 0.5198 - val_loss: 1.1311 - val_accuracy: 0.3947
Epoch 4/100
177/177 [==============================] - 0s 184us/step - loss: 1.0484 - accuracy: 0.5480 - val_loss: 1.3379 - val_accuracy: 0.3553
Epoch 5/100
177/177 [==============================] - 0s 134us/step - loss: 1.1540 - accuracy: 0.5198 - val_loss: 1.2472 - val_accuracy: 0.4737
Epoch 6/100
177/177 [==============================] - 0s 134us/step - loss: 1.0906 - accuracy: 0.5367 - val_loss: 1.0422 - val_accuracy: 0.4868
Epoch 7/100
177/177 [==============================] - 0s 215us/step - loss: 0.9857 -

Epoch 57/100
177/177 [==============================] - 0s 167us/step - loss: 0.5872 - accuracy: 0.8079 - val_loss: 1.0967 - val_accuracy: 0.5000
Epoch 58/100
177/177 [==============================] - 0s 105us/step - loss: 0.5767 - accuracy: 0.7910 - val_loss: 1.1103 - val_accuracy: 0.5000
Epoch 59/100
177/177 [==============================] - 0s 110us/step - loss: 0.5865 - accuracy: 0.7966 - val_loss: 1.1544 - val_accuracy: 0.4474
Epoch 60/100
177/177 [==============================] - 0s 84us/step - loss: 0.5796 - accuracy: 0.7966 - val_loss: 1.0954 - val_accuracy: 0.4737
Epoch 61/100
177/177 [==============================] - 0s 111us/step - loss: 0.5871 - accuracy: 0.7910 - val_loss: 1.1153 - val_accuracy: 0.4737
Epoch 62/100
177/177 [==============================] - 0s 152us/step - loss: 0.5822 - accuracy: 0.8079 - val_loss: 1.1891 - val_accuracy: 0.4342
Epoch 63/100
177/177 [==============================] - 0s 152us/step - loss: 0.5502 - accuracy: 0.8249 - val_loss: 1.1066 - 

In [305]:
acc_test_sel4 = model_sel4.evaluate(X_sel_test, y_sel_test)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_sel4*100))

76/76 [==============================] - 0s 98us/step
over-sampling test accuracy: 55.26%


In [124]:
pred8 = model_sel4.predict_classes(X_sel_test)
pred8

array([2, 1, 0, 1, 2, 0, 1, 2, 1, 0, 0, 1, 2, 1, 0, 2, 1, 2, 0, 1, 2, 0,
       2, 1, 0, 2, 2, 1, 2, 2, 2, 0, 1, 1, 1, 0, 1, 0, 1, 2, 0, 1, 0, 0,
       1, 2, 1, 0, 1, 2, 2, 0, 2, 2, 0, 2, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0])

In [125]:
dat8['pred'] = pred8
dat8

,strain,test,pred
131,NRS104,0,2
123,NRS071,0,1
124,NRS072,1,0
27,BCH-SA-12,0,1
57,CFBREBSa125,2,2
...,...,...,...
170,NRS199,2,1
32,CA11,2,1
212,NRS253,1,1
141,NRS119,2,1


In [126]:
proba8 = model_sel4.predict_proba(X_sel_test)
dat_proba8 = pd.DataFrame(proba8)

In [127]:
dat_proba8

,0,1,2
0,0.045344,0.038492,0.916163
1,0.005553,0.952853,0.041594
2,0.750289,0.023561,0.226149
3,0.232031,0.607860,0.160109
4,0.387979,0.154933,0.457088
...,...,...,...
71,0.160530,0.686465,0.153005
72,0.289323,0.428511,0.282166
73,0.042483,0.848594,0.108923
74,0.066866,0.528038,0.405096


In [128]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [129]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p17ST.csv", index = False,
         header=None)

In [309]:
hist_sel4 = model_sel4.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=100,
          validation_data=(X_sel_test, y_sel_test))

Train on 177 samples, validate on 76 samples
Epoch 1/100
177/177 [==============================] - 0s 102us/step - loss: 0.4224 - accuracy: 0.8644 - val_loss: 1.2902 - val_accuracy: 0.5132
Epoch 2/100
177/177 [==============================] - 0s 88us/step - loss: 0.4583 - accuracy: 0.8418 - val_loss: 1.2461 - val_accuracy: 0.5000
Epoch 3/100
177/177 [==============================] - 0s 88us/step - loss: 0.4816 - accuracy: 0.8362 - val_loss: 1.3103 - val_accuracy: 0.4737
Epoch 4/100
177/177 [==============================] - 0s 111us/step - loss: 0.4068 - accuracy: 0.8475 - val_loss: 1.2924 - val_accuracy: 0.5263
Epoch 5/100
177/177 [==============================] - 0s 151us/step - loss: 0.4207 - accuracy: 0.8418 - val_loss: 1.2429 - val_accuracy: 0.5000
Epoch 6/100
177/177 [==============================] - 0s 124us/step - loss: 0.4034 - accuracy: 0.8531 - val_loss: 1.2574 - val_accuracy: 0.5000
Epoch 7/100
177/177 [==============================] - 0s 129us/step - loss: 0.3752 - a

Epoch 57/100
177/177 [==============================] - 0s 99us/step - loss: 0.3552 - accuracy: 0.8531 - val_loss: 1.4919 - val_accuracy: 0.4342
Epoch 58/100
177/177 [==============================] - 0s 114us/step - loss: 0.3389 - accuracy: 0.8814 - val_loss: 1.5445 - val_accuracy: 0.4605
Epoch 59/100
177/177 [==============================] - 0s 98us/step - loss: 0.3583 - accuracy: 0.8814 - val_loss: 1.5132 - val_accuracy: 0.4605
Epoch 60/100
177/177 [==============================] - 0s 101us/step - loss: 0.3182 - accuracy: 0.8701 - val_loss: 1.5080 - val_accuracy: 0.4474
Epoch 61/100
177/177 [==============================] - 0s 88us/step - loss: 0.3006 - accuracy: 0.8927 - val_loss: 1.5104 - val_accuracy: 0.4474
Epoch 62/100
177/177 [==============================] - 0s 88us/step - loss: 0.3223 - accuracy: 0.9040 - val_loss: 1.5268 - val_accuracy: 0.4342
Epoch 63/100
177/177 [==============================] - 0s 111us/step - loss: 0.3181 - accuracy: 0.9153 - val_loss: 1.5321 - val

In [310]:
print('train accuracy: %.2f%%' % (np.mean(hist_sel4.history['accuracy'])*100))

train accuracy: 87.70%


In [103]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_lasso_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [104]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p0017kpresabs_qual,NRS104,0,1,8.210638e-02,0.917809,8.477923e-05
1,p0017kpresabs_qual,NRS071,0,2,1.015229e-02,0.255327,7.345203e-01
2,p0017kpresabs_qual,NRS072,1,2,3.642946e-03,0.000002,9.963547e-01
3,p0017kpresabs_qual,BCH-SA-12,0,2,2.809318e-03,0.099188,8.980029e-01
4,p0017kpresabs_qual,CFBREBSa125,2,0,4.875667e-01,0.088657,4.237761e-01
...,...,...,...,...,...,...,...
603,p0040presabsSTCC_qual,NRS113,1,2,2.686909e-07,0.093121,9.068785e-01
604,p0040presabsSTCC_qual,BCH-SA-09,2,1,5.529492e-07,0.999990,9.559324e-06
605,p0040presabsSTCC_qual,NRS106,2,1,1.115902e-03,0.998816,6.852559e-05
606,p0040presabsSTCC_qual,CFBREBSa131,2,2,2.708040e-07,0.156768,8.432316e-01


In [105]:
y_prob8 = df_proba8[df_proba8['phage']=='p0017kpresabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[4.53442300e-02, 3.84922800e-02, 9.16163440e-01],
       [5.55331900e-03, 9.52852960e-01, 4.15936200e-02],
       [7.50289500e-01, 2.35614010e-02, 2.26149110e-01],
       [2.32030630e-01, 6.07860270e-01, 1.60109100e-01],
       [3.87979360e-01, 1.54933040e-01, 4.57087550e-01],
       [6.67779270e-01, 2.04682200e-01, 1.27538490e-01],
       [4.35295800e-01, 5.59733150e-01, 4.97103950e-03],
       [1.64425940e-01, 3.35121600e-01, 5.00452460e-01],
       [6.60662100e-02, 7.86608900e-01, 1.47324900e-01],
       [7.18242350e-01, 1.37364760e-01, 1.44392900e-01],
       [4.58425250e-01, 3.15858270e-01, 2.25716460e-01],
       [4.35333900e-02, 6.09235470e-01, 3.47231240e-01],
       [2.44472300e-01, 2.49391570e-01, 5.06136200e-01],
       [2.37668710e-01, 5.71613670e-01, 1.90717640e-01],
       [5.57458460e-01, 2.19031870e-01, 2.23509620e-01],
       [5.13955160e-03, 7.02447200e-02, 9.24615740e-01],
       [1.57207430e-01, 7.52620200e-01, 9.01723500e-02],
       [2.78767530e-01, 3.46911

In [106]:
ovo8 = rocauc_ovo(y_sel_test, y_prob8, average="macro", multi_class="ovo")
ovo8

0.6417806179710941

In [107]:
ovr8 = rocauc_ovr(y_sel_test, y_prob8, average="macro", multi_class="ovr")
ovr8

0.6417806179710941

In [108]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.6282889839437459

In [109]:
np.std(ovos2)

0.024440821586036797

In [110]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.6282889839437459

In [111]:
np.std(ovrs2)

0.024440821586036797

In [311]:
accs_l= [acc_test_sel, acc_test_sel2, acc_test_sel3, acc_test_sel4]

In [312]:
mean_l = np.mean(accs_l)
print('test accuracy mean after lasso: %.2f%%' % (mean_l*100))

test accuracy mean after lasso: 48.36%


In [313]:
std_l = np.std(accs_l)
print('test accuracy standard deviation after lasso:', std_l)

test accuracy standard deviation after lasso: 0.052115058683581295


In [314]:
accs_train_l = [np.mean(hist_sel.history['accuracy']), np.mean(hist_sel2.history['accuracy']), np.mean(hist_sel3.history['accuracy']),
             np.mean(hist_sel4.history['accuracy'])]

In [315]:
mean_train_l = np.mean(accs_train_l)
print('train accuracy mean after lasso: %.2f%%' % (mean_train_l*100))

train accuracy mean after lasso: 88.01%


In [316]:
std_train_l = np.std(accs_train_l)
print('train accuracy standard deviation after lasso:', std_train_l)

train accuracy standard deviation after lasso: 0.010038472
